# Agent 4: DRAFTER AI Agent

In [2]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage

from langchain_groq import ChatGroq
from langchain_core.tools import tool

from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

from langgraph.prebuilt import ToolNode

load_dotenv()

True